# LCEL

https://python.langchain.com/docs/expression_language/

In [9]:
# get api key
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

## Chapter 7: OutputParsers and Memory

## 7.1 Parsers

In [2]:
# 7.1.1
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_openai.llms import OpenAI

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

prompt_template = """
Suggest some names for my {subject} startup. \n
{format_instructions}
"""

prompt = PromptTemplate.from_template(prompt_template)

llm = OpenAI()

chain = prompt | llm | output_parser

print(
    chain.invoke(
        {
            "format_instructions": format_instructions,
            "subject": "Mobile",
        }
    )
)

['1. Swiftphone\n2. NexGen Mobile\n3. OnTheGo Tech\n4. PocketTech\n5. Infinity Mobile\n6. SmartLink\n7. MobileMate\n8. GoMobile\n9. iConnect\n10. Skyphone\n11. TechTrail\n12. MobileWave\n13. NetEdge Mobile\n14. ReachOut Tech\n15. MobileFusion\n16. NextMobile\n17. UpLink Tech\n18. MobileSphere\n19. BoostTech\n20. CellSpark']


In [32]:
# 7.1.2 Custom OutputParser
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from langchain_openai.llms import OpenAI
from langchain.output_parsers import OutputFixingParser
from operator import itemgetter
from langchain.schema.runnable import RunnablePassthrough

response_schemas = [
    ResponseSchema(
        name="Monument",
        description="The monument mentioned in the answer.",
    ),
    ResponseSchema(
        name="city", description="the city in which this monument is present."
    ),
    ResponseSchema(
        name="architect",
        description="the architect of the monument.",
    ),
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"Monument": string  // The monument mentioned in the answer.
	"city": string  // the city in which this monument is present.
	"architect": string  // the architect of the monument.
}
```


In [39]:
prompt_template = """
    answer the users question as best as possible. \n
    {format_instructions}\n {question}
"""

prompt = PromptTemplate.from_template(prompt_template)

llm = OpenAI()

chain = prompt | llm | output_parser

print(
    chain.invoke(
        {
            "format_instructions": format_instructions,
            "question": "Choose a random Indian state and pen down some cities",
        }
    )
)

{'Monument': 'Taj Mahal', 'city': 'Agra', 'architect': 'Ustad Ahmad Lahauri'}


In [11]:
# 7.13

from langchain.output_parsers import PydanticOutputParser
from langchain.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from typing import List
from langchain_openai import OpenAI


class State(BaseModel):
    state: str = Field(description="name of an Indian State")
    cities: List[str] = Field(description="Indian cities in that state")


parser = PydanticOutputParser(pydantic_object=State)

format_instructions = parser.get_format_instructions()

template = """answer the users question as best as possible and generate output. \n
{format_instructions} \n{questions}
"""

prompt = PromptTemplate.from_template(template)
llm = OpenAI(temperature=0, api_key=OPENAI_API_KEY)
chain = prompt | llm | parser

chain.invoke(
    {
        "questions": "State is Andhra Pradesh",
        "format_instructions": format_instructions,
    }
)

State(state='Andhra Pradesh', cities=['Visakhapatnam', 'Vijayawada', 'Guntur', 'Nellore', 'Kurnool', 'Rajahmundry', 'Kakinada', 'Tirupati', 'Anantapur', 'Kadapa'])

## 7.2 Memory

In [12]:
# coversational buffer memory
from langchain.chains import LLMChain
from langchain_openai import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
import os

In [17]:
template = """You are a chatbot having a conversation with a human.
{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate.from_template(template)
memory = ConversationBufferMemory(memory_key="chat_history")
llm = OpenAI(api_key=OPENAI_API_KEY, temperature=0)

chain = LLMChain(
    prompt=prompt,
    memory=memory,
    llm=llm,
    verbose=True,
)

chain.invoke("hi there, my name is Raghu")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.

Human: hi there, my name is Raghu
Chatbot:

> Finished chain.


{'human_input': 'hi there, my name is Raghu',
 'chat_history': '',
 'text': " Hello Raghu, it's nice to meet you. I am a chatbot designed to have conversations with humans. How can I assist you today?"}

In [18]:
chain.invoke("write a song about me")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.
Human: hi there, my name is Raghu
AI:  Hello Raghu, it's nice to meet you. I am a chatbot designed to have conversations with humans. How can I assist you today?
Human: write a song about me
Chatbot:

> Finished chain.


{'human_input': 'write a song about me',
 'chat_history': "Human: hi there, my name is Raghu\nAI:  Hello Raghu, it's nice to meet you. I am a chatbot designed to have conversations with humans. How can I assist you today?",
 'text': " Sure, Raghu, here's a little song I came up with just for you:\n\nVerse 1:\nRaghu, oh Raghu, with a heart of gold\nYour smile shines bright, never growing old\nYou light up the room with your infectious laugh\nAnd your kindness and warmth, they're hard to surpass\n\nChorus:\nOh Raghu, you're one of a kind\nA true gem, so hard to find\nYour presence brings joy, wherever you go\nYou're a shining star, don't you know?\n\nVerse 2:\nWith your passion and drive, you reach for the stars\nAnd your determination, it knows no bars\nYou inspire those around you, to be their best\nAnd with you by their side, they feel truly blessed\n\nChorus:\nOh Raghu, you're one of a kind\nA true gem, so hard to find\nYour presence brings joy, wherever you go\nYou're a shining star

In [19]:
chain.invoke("what is the song about?")



> Entering new LLMChain chain...
Prompt after formatting:
You are a chatbot having a conversation with a human.
Human: hi there, my name is Raghu
AI:  Hello Raghu, it's nice to meet you. I am a chatbot designed to have conversations with humans. How can I assist you today?
Human: write a song about me
AI:  Sure, Raghu, here's a little song I came up with just for you:

Verse 1:
Raghu, oh Raghu, with a heart of gold
Your smile shines bright, never growing old
You light up the room with your infectious laugh
And your kindness and warmth, they're hard to surpass

Chorus:
Oh Raghu, you're one of a kind
A true gem, so hard to find
Your presence brings joy, wherever you go
You're a shining star, don't you know?

Verse 2:
With your passion and drive, you reach for the stars
And your determination, it knows no bars
You inspire those around you, to be their best
And with you by their side, they feel truly blessed

Chorus:
Oh Raghu, you're one of a kind
A true gem, so hard to find
Your presenc

{'human_input': 'what is the song about?',
 'chat_history': "Human: hi there, my name is Raghu\nAI:  Hello Raghu, it's nice to meet you. I am a chatbot designed to have conversations with humans. How can I assist you today?\nHuman: write a song about me\nAI:  Sure, Raghu, here's a little song I came up with just for you:\n\nVerse 1:\nRaghu, oh Raghu, with a heart of gold\nYour smile shines bright, never growing old\nYou light up the room with your infectious laugh\nAnd your kindness and warmth, they're hard to surpass\n\nChorus:\nOh Raghu, you're one of a kind\nA true gem, so hard to find\nYour presence brings joy, wherever you go\nYou're a shining star, don't you know?\n\nVerse 2:\nWith your passion and drive, you reach for the stars\nAnd your determination, it knows no bars\nYou inspire those around you, to be their best\nAnd with you by their side, they feel truly blessed\n\nChorus:\nOh Raghu, you're one of a kind\nA true gem, so hard to find\nYour presence brings joy, wherever you 

In [20]:
memory.load_memory_variables({})

{'chat_history': "Human: hi there, my name is Raghu\nAI:  Hello Raghu, it's nice to meet you. I am a chatbot designed to have conversations with humans. How can I assist you today?\nHuman: write a song about me\nAI:  Sure, Raghu, here's a little song I came up with just for you:\n\nVerse 1:\nRaghu, oh Raghu, with a heart of gold\nYour smile shines bright, never growing old\nYou light up the room with your infectious laugh\nAnd your kindness and warmth, they're hard to surpass\n\nChorus:\nOh Raghu, you're one of a kind\nA true gem, so hard to find\nYour presence brings joy, wherever you go\nYou're a shining star, don't you know?\n\nVerse 2:\nWith your passion and drive, you reach for the stars\nAnd your determination, it knows no bars\nYou inspire those around you, to be their best\nAnd with you by their side, they feel truly blessed\n\nChorus:\nOh Raghu, you're one of a kind\nA true gem, so hard to find\nYour presence brings joy, wherever you go\nYou're a shining star, don't you know?\

# 7.2.2 ConversationSummaryMemory